# Libraries

In [26]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import demoji
from tqdm import tqdm
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Data Preperation

#### Peek Data

In [9]:
df = pd.read_csv("Suicide_Detection.csv")
df.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


#### Class Balance

In [10]:
df["class"].value_counts()

suicide        116037
non-suicide    116037
Name: class, dtype: int64

Suicide and non-suicide text are equal in number.

#### Add space to some camel cased words

In [11]:
df['text'] = df['text'].replace(r"(\w)([A-Z])", r"\1 \2", regex=True)
df['text']

0         Ex Wife Threatening Suicide Recently I left my...
1         Am I weird I don't get affected by compliments...
2         Finally 2020 is almost over... So I can never ...
3                 i need helpjust help me im crying so hard
4         I’m so lost Hello, my name is Adam (16) and I’...
                                ...                        
232069    If you don't like rock then your not going to ...
232070    You how you can tell i have so many friends an...
232071    pee probably tastes like salty tea😏💦‼️ can som...
232072    The usual stuff you find here I'm not posting ...
232073    I still haven't beaten the first boss in Hollo...
Name: text, Length: 232074, dtype: object

#### Convert Emojis into Text

In [12]:
def emoji_to_text(text):
    return demoji.replace_with_desc(text, " ")

In [13]:
tqdm.pandas(desc="Progress Bar: ")

df["text"] = df["text"].progress_apply(emoji_to_text)

Progress Bar: 100%|███████████████████████████████████████████████████████████| 232074/232074 [24:16<00:00, 159.29it/s]


In [14]:
df["text"]

0         Ex Wife Threatening Suicide Recently I left my...
1         Am I weird I don't get affected by compliments...
2         Finally 2020 is almost over... So I can never ...
3                 i need helpjust help me im crying so hard
4         I’m so lost Hello, my name is Adam (16) and I’...
                                ...                        
232069    If you don't like rock then your not going to ...
232070    You how you can tell i have so many friends an...
232071    pee probably tastes like salty tea smirking fa...
232072    The usual stuff you find here I'm not posting ...
232073    I still haven't beaten the first boss in Hollo...
Name: text, Length: 232074, dtype: object

#### Lowercase alphabets

In [15]:
df["text"] = df["text"].str.lower()

In [16]:
df["text"]

0         ex wife threatening suicide recently i left my...
1         am i weird i don't get affected by compliments...
2         finally 2020 is almost over... so i can never ...
3                 i need helpjust help me im crying so hard
4         i’m so lost hello, my name is adam (16) and i’...
                                ...                        
232069    if you don't like rock then your not going to ...
232070    you how you can tell i have so many friends an...
232071    pee probably tastes like salty tea smirking fa...
232072    the usual stuff you find here i'm not posting ...
232073    i still haven't beaten the first boss in hollo...
Name: text, Length: 232074, dtype: object

#### Remove Punctuations

In [17]:
# Not alphabet o digit, remove
df["text"] = df['text'].str.replace('[^\w\s]','')

C:\Users\yeohz\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [19]:
df["text"]

0         ex wife threatening suicide recently i left my...
1         am i weird i dont get affected by compliments ...
2         finally 2020 is almost over so i can never hea...
3                 i need helpjust help me im crying so hard
4         im so lost hello my name is adam 16 and ive be...
                                ...                        
232069    if you dont like rock then your not going to g...
232070    you how you can tell i have so many friends an...
232071    pee probably tastes like salty tea smirking fa...
232072    the usual stuff you find here im not posting t...
232073    i still havent beaten the first boss in hollow...
Name: text, Length: 232074, dtype: object

#### Remove Stop Words

In [30]:
# Show incuded stop words
nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yeohz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: 'list' object has no attribute 'words'

In [33]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [36]:
stopWordExceptions = ["but", "if", "because", "against",'no', 'nor','not', 'don', "don't", "ain", 
"aren't", 'aren', " 'couldn'",  "couldn't", 'didn',"didn't", "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
"haven't", 'isn', "isn't", "mustn't", 'shan', "shan't", 'shouldn',"shouldn't", 'wasn', "wasn't",
'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [43]:
print("Stop word exceptions: ", len(stopWordExceptions))
print("Stop words: ", len(stopwords))

Stop word exceptions:  38
Stop words:  179


In [44]:
customStopWords =[word for word in stopwords if word not in stopWordExceptions]
len(customStopWords)

142

# Models & Evaluation